#**Машинное обучение ИБ-2024**

#**Домашнее задание 1.**
#Регрессия, KNN, LinearRegression.

В данной домашней работе мы будем строить модели для предсказания цены квартиры в России. Ниже приведено описание некоторых колонок набора данных.

date - дата публикации объявления

price - цена в рублях

level- этаж, на котором находится квартира

levels - количество этажей в квартире

rooms - количество комнат в квартире. Если значение -1, то квартира считается апартаментами.

area - площадь квартиры.

kitchen_area - площадь кухни.

geo_lat - Latitude

geo_lon - Longitude

building_type - материал застройки. 0 - Don't know. 1 - Other. 2 - Panel. 3 - Monolithic. 4 - Brick. 5 - Blocky. 6 - Wooden

#Часть 0. Начало работы

Для начала работы с данными импортируем библиотеки, которые понадобятся в данном задании.

In [119]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression as SkLinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

Загрузим библиотеку folium для отображения данных на карте по координатам.

In [120]:
#!pip install folium

Загрузим данные из csv файла в датафрейм.

In [121]:
DataFrame = pd.read_csv('input_data.csv', sep=';')

# Выводим информацию о загруженных данных
print(f"Загрузка данных выполнена. Размер DataFrame: {DataFrame.shape}")
print(f"Список столбцов: {DataFrame.columns.tolist()}")

Загрузка данных выполнена. Размер DataFrame: (11358150, 15)
Список столбцов: ['date', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area', 'geo_lat', 'geo_lon', 'building_type', 'object_type', 'postal_code', 'street_id', 'id_region', 'house_id']


In [122]:
print("=== Первые 5 строк DataFrame ===")
print(DataFrame.head())

# Выводим общую информацию о данных
print("\n=== Информация о DataFrame ===")
print(DataFrame.info())

# Выводим общее количество строк и количество строк, где building_type < 1
total_rows = DataFrame.shape[0]
filtered_rows = DataFrame[DataFrame["building_type"] < 1].shape[0]
print(f"\nОбщее количество строк: {total_rows}")
print(f"Количество строк с building_type < 1: {filtered_rows}")

=== Первые 5 строк DataFrame ===
         date     price  level  levels  rooms  area  kitchen_area    geo_lat  \
0  2021-01-01   2451300     15      31      1  30.3           0.0  56.780112   
1  2021-01-01   1450000      5       5      1  33.0           6.0  44.608154   
2  2021-01-01  10700000      4      13      3  85.0          12.0  55.540060   
3  2021-01-01   3100000      3       5      3  82.0           9.0  44.608154   
4  2021-01-01   2500000      2       3      1  30.0           9.0  44.738685   

     geo_lon  building_type  object_type  postal_code  street_id  id_region  \
0  60.699355              0            2     620000.0        NaN         66   
1  40.138381              0            0     385000.0        NaN          1   
2  37.725112              3            0     142701.0   242543.0         50   
3  40.138381              0            0     385000.0        NaN          1   
4  37.713668              3            2     353960.0   439378.0         23   

    house_i

Отобразим на карте координаты наших построек.

In [123]:
import folium
from IPython.display import display

map_DataFrame = DataFrame.head(1000).copy()

m = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

# Добавляем точки на карту
for idx, row in map_DataFrame.iterrows():
    folium.Marker(
        location=[row['geo_lat'], row['geo_lon']],
        popup=f"Price: {row['price']} rub"
    ).add_to(m)

display(m)

# Часть 1. Подготовим данные для обработки моделями машинного обучения.

**0.5 Балл**. География наших наблюдений в наборе данных крайне большая. Однако мы знаем, что стоимость квартир в Москве и Санкт-Петербурге намного выше, чем в среднем по России. Давайте сделаем признаки, который показывают, находится ли квартира в 20 килиметрах от центра Москвы или находится ли квартира в 20 килиметрах от центра Санкт-Петербурга.

Создайте два признака is_Moscow и is_Saint_Peterburg. Для нахождения расстояния по координатам используйте функцию haversine_distance.

In [124]:
def haversine_distance(lat1, lon1, lat2, lon2):
    # Радиус Земли в километрах
    EARTH_RADIUS = 6371
    
    # Преобразуем координаты в радианы
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Разница координат
    delta_lat = lat2 - lat1
    delta_lon = lon2 - lon1
    
    # Формула Хаверсина для вычисления расстояния
    a = np.sin(delta_lat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(delta_lon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    return EARTH_RADIUS * c

# Координаты центров городов (широта, долгота)
MOSCOW_CENTER = (55.7558, 37.6173)  # Москва
SPB_CENTER = (59.9343, 30.3351)     # Санкт-Петербург

print("=== Создание географических признаков ===")

# Вычисляем расстояние до центра Москвы и создаем признак is_Moscow
DataFrame['distance_to_Moscow'] = haversine_distance(
    DataFrame['geo_lat'], DataFrame['geo_lon'], 
    MOSCOW_CENTER[0], MOSCOW_CENTER[1]
)
DataFrame['is_Moscow'] = DataFrame['distance_to_Moscow'] <= 20

# Вычисляем расстояние до центра Санкт-Петербурга и создаем признак is_Saint_Petersburg
DataFrame['distance_to_spb'] = haversine_distance(
    DataFrame['geo_lat'], DataFrame['geo_lon'], 
    SPB_CENTER[0], SPB_CENTER[1]
)
DataFrame['is_Saint_Petersburg'] = DataFrame['distance_to_spb'] <= 20

# Выводим информацию о созданных признаках
print("Создание признаков завершено: is_Moscow, is_Saint_Petersburg")
print(f"Количество квартир в Москве (в радиусе 20 км): {DataFrame['is_Moscow'].sum()}")
print(f"Количество квартир в Санкт-Петербурге (в радиусе 20 км): {DataFrame['is_Saint_Petersburg'].sum()}")

=== Создание географических признаков ===
Создание признаков завершено: is_Moscow, is_Saint_Petersburg
Количество квартир в Москве (в радиусе 20 км): 1027378
Количество квартир в Санкт-Петербурге (в радиусе 20 км): 875985


**0.5 Балла**. В нашем наборе данных есть признаки, которые мы теоретически можем использовать, например postal_code, но мы это будем делать в рамках домашней работы очень-очень долго. Поэтому предлагается удалить ненужные признаки из датафрейма.

Удалим geo_lat,	geo_lon,	object_type,	postal_code,	street_id,	id_region,	house_id.

In [125]:
# Список столбцов для удаления
columns_to_drop = ['geo_lat', 'geo_lon', 'object_type', 'postal_code', 
                   'street_id', 'id_region', 'house_id', 'distance_to_Moscow', 'distance_to_spb']

# Фильтруем только те столбцы, которые существуют в DataFrame
valid_columns_to_drop = [col for col in columns_to_drop if col in DataFrame.columns]

# Выводим информацию об удаляемых столбцах
print(f"Удаляемые столбцы: {valid_columns_to_drop}")

# Удаляем столбцы из DataFrame
DataFrame = DataFrame.drop(columns=valid_columns_to_drop)

# Выводим список оставшихся столбцов
print(f"Оставшиеся столбцы: {DataFrame.columns.tolist()}")

Удаляемые столбцы: ['geo_lat', 'geo_lon', 'object_type', 'postal_code', 'street_id', 'id_region', 'house_id', 'distance_to_Moscow', 'distance_to_spb']
Оставшиеся столбцы: ['date', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area', 'building_type', 'is_Moscow', 'is_Saint_Petersburg']


In [126]:
def clean_dataframe(DataFrame):
    # Создаем копию DataFrame для безопасной обработки
    DataFrame_clean = DataFrame.copy()
    
    # Удаляем строки с площадью менее 10 кв.м
    DataFrame_clean = DataFrame_clean[DataFrame_clean['area'] > 10]
    
    # Фильтруем цены в разумном диапазоне (от 100 тыс. до 500 млн)
    DataFrame_clean = DataFrame_clean[(DataFrame_clean['price'] > 100_000) & (DataFrame_clean['price'] < 500_000_000)]
    
    # Удаляем строки, где этаж превышает общее число этажей
    DataFrame_clean = DataFrame_clean[DataFrame_clean['level'] <= DataFrame_clean['levels']]
    
    # Удаляем строки с нулевым или отрицательным числом этажей
    DataFrame_clean = DataFrame_clean[DataFrame_clean['levels'] > 0]
    
    return DataFrame_clean

# Применяем функцию очистки к данным
DataFrame_clean = clean_dataframe(DataFrame)

**0.5 Балл**. Для начала Вам предлагается проанализировать Ваши оставшиеся признаки (колонки) в наборе данных. Какие колонки категориальные? Какие числовые?

Категориальные: rooms, building_type, is_Moscow, is_Saint_Petersburg

Числовые: price, level, levels, area, kitchen_area

Давайте закодируем категориальные признаки с помощью OneHot-Encoding. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [127]:
# Выводим заголовок для наглядности — начинаем анализ типов данных в датафрейме
print("Анализ типов данных по заданию:")
# Выводим типы данных всех столбцов в датафрейме DataFrame (например, int64, object, float64 и т.д.)
print(DataFrame.dtypes)

# Инициализируем два пустых списка для хранения имён категориальных и числовых признаков
categorical_cols = []   # Список для категориальных признаков
numerical_cols = []     # Список для числовых признаков

# Проходим по всем столбцам датафрейма
for col in DataFrame.columns:
    # Пропускаем целевую переменную 'price', так как она не используется как признак
    if col == 'price':
        continue
    # Согласно заданию, 'building_type' — единственный явно указанный категориальный признак
    elif col == 'building_type':
        categorical_cols.append(col)  # Добавляем его в список категориальных
        # Выводим информацию: имя признака и количество уникальных значений (для понимания разнообразия категорий)
        print(f"Категориальный: {col} - {DataFrame[col].nunique()} уникальных значений")
    else:
        # Все остальные признаки (кроме 'price' и 'building_type') считаем числовыми
        numerical_cols.append(col)
        print(f"Числовой: {col}")

# Выводим итоговую сводку по типам признаков
print(f"\nИТОГО:")
print(f"Категориальные признаки для кодирования: {categorical_cols}")
print(f"Числовые признаки: {numerical_cols}")

# Проверяем, есть ли категориальные признаки, требующие кодирования
if categorical_cols:
    print(f"\nПрименяем One-Hot Encoding к: {categorical_cols}")
    
    # Создаём экземпляр OneHotEncoder из sklearn
    # drop='first' — удаляем один из фиктивных признаков, чтобы избежать мультиколлинеарности (ловушки фиктивной переменной)
    # sparse_output=False — возвращаем плотный массив (DataFrame), а не разреженную матрицу
    encoder = OneHotEncoder(drop='first', sparse_output=False)
    
    # Применяем обучение и трансформацию только к столбцам из categorical_cols (в данном случае — только 'building_type')
    encoded_cols = encoder.fit_transform(DataFrame[categorical_cols])
    
    # Преобразуем результат в pandas DataFrame с осмысленными названиями столбцов
    # encoder.get_feature_names_out(categorical_cols) генерирует имена новых бинарных признаков,
    # например: 'building_type_1', 'building_type_2' и т.д.
    encoded_DataFrame = pd.DataFrame(
        encoded_cols, 
        columns=encoder.get_feature_names_out(categorical_cols)
    )
    
    # Удаляем исходные категориальные столбцы из DataFrame и добавляем новые закодированные столбцы
    # axis=1 означает объединение по столбцам (горизонтально)
    DataFrame = pd.concat([DataFrame.drop(columns=categorical_cols), encoded_DataFrame], axis=1)
    
    # Выводим информацию о результатах кодирования
    print(f"После кодирования. Размер: {DataFrame.shape}")  # Новый размер датафрейма
    # Показываем, какие новые столбцы 
    print(f"Новые колонки от OneHotEncoder: {[col for col in DataFrame.columns if 'building_type' in col]}")
else:
    # Если категориальных признаков нет, выводим 
    print("Нет категориальных признаков для кодирования")

# Выводим полный список колонок в итоговом датафрейме
print(f"\nИтоговые колонки: {DataFrame.columns.tolist()}")

Анализ типов данных по заданию:
date                    object
price                    int64
level                    int64
levels                   int64
rooms                    int64
area                   float64
kitchen_area           float64
building_type            int64
is_Moscow                 bool
is_Saint_Petersburg       bool
dtype: object
Числовой: date
Числовой: level
Числовой: levels
Числовой: rooms
Числовой: area
Числовой: kitchen_area
Категориальный: building_type - 7 уникальных значений
Числовой: is_Moscow
Числовой: is_Saint_Petersburg

ИТОГО:
Категориальные признаки для кодирования: ['building_type']
Числовые признаки: ['date', 'level', 'levels', 'rooms', 'area', 'kitchen_area', 'is_Moscow', 'is_Saint_Petersburg']

Применяем One-Hot Encoding к: ['building_type']
После кодирования. Размер: (11358150, 15)
Новые колонки от OneHotEncoder: ['building_type_1', 'building_type_2', 'building_type_3', 'building_type_4', 'building_type_5', 'building_type_6']

Итоговые колонки

**0.5 Балл**. Поработаем с числовыми признаками:


1.   Добавьте в ваш датасет два признака: количество дней со дня первого наблюдения (разница между датами объявлений). Возможно, для предсказания цены не так важен этаж, как важно отношение этажа квартиры на количество этажей в доме, добавьте этот признак. После добавления нового признака колонку date можно удалить.
2.   Числовые признаки могут иметь разные порядки. Давайте отнормируем числовые признаки с помощью StandartScaller https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.



In [128]:
print("=== Обработка числовых признаков ===")

# 1. Обработка столбца с датой, если он существует
if 'date' in DataFrame.columns:
    print("1. Обработка столбца 'date'...")
    # Преобразуем столбец 'date' в формат datetime
    DataFrame['date'] = pd.to_datetime(DataFrame['date'])
    # Вычисляем количество дней от минимальной даты
    earliest_date = DataFrame['date'].min()
    DataFrame['days_from_first'] = (DataFrame['date'] - earliest_date).dt.days
    # Удаляем исходный столбец 'date'
    DataFrame = DataFrame.drop(columns=['date'])
    print("Создан признак 'days_from_first'. Столбец 'date' удален")
else:
    print("1. Столбец 'date' отсутствует, пропускаем обработку даты")

# Выводим первые 5 строк после обработки даты
print("\nДанные после обработки даты:")
print(DataFrame.head())

# 2. Создание признака отношения этажа к общему числу этажей
print("\n2. Создание признака 'level_ratio'...")
if 'level' in DataFrame.columns and 'levels' in DataFrame.columns:
    # Вычисляем отношение этажа к общему числу этажей, избегая деления на ноль
    DataFrame['level_ratio'] = np.where(DataFrame['levels'] != 0, DataFrame['level'] / DataFrame['levels'], 0)
    print("Создан признак 'level_ratio'")
    # Удаляем исходные столбцы 'level' и 'levels'
    columns_to_drop = ['level', 'levels']
    DataFrame = DataFrame.drop(columns=columns_to_drop)
    print("Столбцы 'level' и 'levels' удалены")
else:
    missing_cols = [col for col in ['level', 'levels'] if col not in DataFrame.columns]
    print(f"   Предупреждение: отсутствуют столбцы {missing_cols}")

# Выводим первые 5 строк после создания level_ratio
print("\nДанные после создания 'level_ratio':")
print(DataFrame.head())

# 3. Подготовка данных для моделирования
print("\n3. Подготовка данных для моделирования...")

# Определяем числовые признаки для нормализации, исключая 'price' и бинарные признаки
numerical_features_to_scale = [
    col for col in DataFrame.columns 
    if col != 'price' 
    and DataFrame[col].dtype in ['int64', 'float64', 'int32']
    and 'building_type' not in col
    and col not in ['is_Moscow', 'is_Saint_Petersburg']
]

print(f"Числовые признаки для нормализации: {numerical_features_to_scale}")

# Разделяем данные на признаки (X) и целевую переменную (y)
X = DataFrame.drop(columns=['price'])
y = DataFrame['price']

print(f"Размерность матрицы признаков: {X.shape}")
print(f"Размерность целевой переменной: {y.shape}")

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=28
)

print(f"Размер обучающей выборки: {X_train.shape}")
print(f"Размер тестовой выборки: {X_test.shape}")

# 4. Нормализация числовых признаков
print("\n4. Нормализация числовых признаков...")
if numerical_features_to_scale:
    # Инициализируем StandardScaler
    scaler = StandardScaler()
    
    # Нормализуем числовые признаки в обучающей выборке
    X_train[numerical_features_to_scale] = scaler.fit_transform(X_train[numerical_features_to_scale])
    
    # Применяем те же параметры нормализации к тестовой выборке
    X_test[numerical_features_to_scale] = scaler.transform(X_test[numerical_features_to_scale])
    
    print("Нормализация выполнена успешно!")
    print("Статистика после нормализации (для первых трех признаков):")
    for col in numerical_features_to_scale[:3]:
        print(f"  {col}: среднее={X_train[col].mean():.2f}, стд. отклонение={X_train[col].std():.2f}")
else:
    print("   Числовые признаки для нормализации отсутствуют")

# Выводим итоговые размеры данных и столбцы
print(f"\nИтоговый размер обучающей выборки: {X_train.shape}")
print(f"Итоговый размер тестовой выборки: {X_test.shape}")
print(f"Итоговые столбцы: {X_train.columns.tolist()}")

# Показываем первые 5 строк обучающих данных
print("\nПервые 5 строк обучающих данных после обработки:")
print(X_train.head())

=== Обработка числовых признаков ===
1. Обработка столбца 'date'...
Создан признак 'days_from_first'. Столбец 'date' удален

Данные после обработки даты:
      price  level  levels  rooms  area  kitchen_area  is_Moscow  \
0   2451300     15      31      1  30.3           0.0      False   
1   1450000      5       5      1  33.0           6.0      False   
2  10700000      4      13      3  85.0          12.0      False   
3   3100000      3       5      3  82.0           9.0      False   
4   2500000      2       3      1  30.0           9.0      False   

   is_Saint_Petersburg  building_type_1  building_type_2  building_type_3  \
0                False              0.0              0.0              0.0   
1                False              0.0              0.0              0.0   
2                False              0.0              0.0              1.0   
3                False              0.0              0.0              0.0   
4                False              0.0             

**2 Балла**. Реализуйте класс KNNRegressor, который должен делать регрессию методом k ближайших соседей.

In [129]:
import numpy as np
from numba import njit, prange

@njit(parallel=True, fastmath=True)
def calculate_euclidean_distances(X1, X2):
    n1, n2 = X1.shape[0], X2.shape[0]
    distances = np.empty((n1, n2), dtype=np.float32)
    
    for i in prange(n1):
        for j in range(n2):
            sum_squares = 0.0
            for k in range(X1.shape[1]):
                diff = X1[i, k] - X2[j, k]
                sum_squares += diff * diff
            distances[i, j] = np.sqrt(sum_squares)
    
    return distances

class KNNRegressor:
    def __init__(self, n_neighbors=5, batch_size=5000):
        self.k = n_neighbors
        self.batch_size = batch_size

    def fit(self, X, y):
        # Преобразуем входные данные в NumPy массивы с типом float32
        self.X_train = X.to_numpy(dtype=np.float32) if not isinstance(X, np.ndarray) else X.astype(np.float32)
        self.y_train = y.to_numpy(dtype=np.float32) if not isinstance(y, np.ndarray) else y.astype(np.float32)

    def predict(self, X):
        X = X.to_numpy(dtype=np.float32) if not isinstance(X, np.ndarray) else X.astype(np.float32)
        
        n_samples = X.shape[0]
        predictions = np.zeros(n_samples, dtype=np.float32)

        # Обрабатываем данные батчами
        for start in range(0, n_samples, self.batch_size):
            end = min(start + self.batch_size, n_samples)
            X_batch = X[start:end]

            # Вычисляем евклидовы расстояния для текущего батча
            distances = calculate_euclidean_distances(X_batch, self.X_train)

            # Находим индексы k ближайших соседей
            nearest_indices = np.argpartition(distances, self.k, axis=1)[:, :self.k]
            
            # Вычисляем среднее значение целевой переменной для k соседей
            predictions[start:end] = np.mean(self.y_train[nearest_indices], axis=1)

        return predictions

**3 Балла**. Реализуйте класс LinearRegression, поддерживающий обучение градиентными спусками SGD, Momentum, AdaGrad. Используйте градиент для оптимизации функции потерь MSE.

In [130]:
class LinearRegression:
    def __init__(self, learning_rate=0.01, optimization='SGD', max_iter=1000, tolerance=1e-4):
        self.learning_rate = learning_rate
        self.optimization = optimization
        self.max_iter = max_iter
        self.tolerance = tolerance
        self.weights = None
        self.bias = None
        self.loss_history = []
        self.X_mean = None
        self.X_std = None
        self.y_mean = None
        self.y_std = None

    def fit(self, X, y):
        # Преобразуем входные данные в NumPy массивы
        X = X.values if isinstance(X, pd.DataFrame) else X
        y = y.values if isinstance(y, pd.Series) else y

        # Нормализация признаков
        self.X_mean = np.mean(X, axis=0)
        self.X_std = np.std(X, axis=0) + 1e-8  # Добавляем эпсилон для избежания деления на ноль
        X_normalized = (X - self.X_mean) / self.X_std

        # Нормализация целевой переменной
        self.y_mean = np.mean(y)
        self.y_std = np.std(y) + 1e-8
        y_normalized = (y - self.y_mean) / self.y_std

        n_samples, n_features = X_normalized.shape

        # Инициализация весов и смещения
        self.weights = np.zeros(n_features)
        self.bias = 0.0

        # Инициализация параметров для оптимизаторов
        if self.optimization == 'Momentum':
            self.velocity_w = np.zeros(n_features)
            self.velocity_b = 0.0
        elif self.optimization == 'AdaGrad':
            self.grad_sum_w = np.zeros(n_features) + 1e-8
            self.grad_sum_b = 1e-8

        previous_loss = float('inf')

        # Обучение с использованием градиентного спуска
        for iteration in range(self.max_iter):
            # Прямое распространение
            y_pred = np.dot(X_normalized, self.weights) + self.bias

            # Вычисляем градиенты
            error = y_pred - y_normalized
            grad_w = np.dot(X_normalized.T, error) / n_samples
            grad_b = np.sum(error) / n_samples

            # Клиппинг градиентов для стабильности
            grad_w = np.clip(grad_w, -1.0, 1.0)
            grad_b = np.clip(grad_b, -1.0, 1.0)

            # Уменьшение скорости обучения (learning rate decay)
            current_lr = self.learning_rate / (1 + 0.001 * iteration)

            # Обновление параметров в зависимости от метода оптимизации
            if self.optimization == 'SGD':
                self.weights -= current_lr * grad_w
                self.bias -= current_lr * grad_b
            elif self.optimization == 'Momentum':
                self.velocity_w = 0.9 * self.velocity_w + current_lr * grad_w
                self.velocity_b = 0.9 * self.velocity_b + current_lr * grad_b
                self.weights -= self.velocity_w
                self.bias -= self.velocity_b
            elif self.optimization == 'AdaGrad':
                self.grad_sum_w += grad_w**2
                self.grad_sum_b += grad_b**2
                self.weights -= current_lr * grad_w / np.sqrt(self.grad_sum_w)
                self.bias -= current_lr * grad_b / np.sqrt(self.grad_sum_b)

            # Вычисляем и сохраняем функцию потерь (MSE)
            loss = np.mean(error**2)
            self.loss_history.append(loss)

            # Проверяем условие остановки
            if abs(previous_loss - loss) < self.tolerance:
                break
            previous_loss = loss

    def predict(self, X):
        # Преобразуем входные данные в NumPy массив
        X = X.values if isinstance(X, pd.DataFrame) else X

        # Нормализуем входные данные
        X_normalized = (X - self.X_mean) / self.X_std

        # Вычисляем предсказания в нормализованном пространстве
        y_pred_normalized = np.dot(X_normalized, self.weights) + self.bias

        # Денормализуем предсказания
        return y_pred_normalized * self.y_std + self.y_mean

# Часть 2. Эксперименты с моделями машинного обучения.

**3 Балла**. Проведите эксперименты с написанными Вами методами машинного обучения. Выделите обучающую и тестовую выборки в отношении 0,8 и 0,2 соответственно. Измерьте ошибку MSE, MAE, RMSE. Заиспользуйте методы KNNRegressor и LinearRegression из библиотеки sklearn, сравните качество Ваших решений и библиотечных.

In [ ]:
# Логарифмирование целевой переменной
y_train_log = np.log1p(y_train)  # Применяем log(1 + x) к обучающей выборке
y_test_log = np.log1p(y_test)    # Применяем log(1 + x) к тестовой выборке

def super_clean_dataframe(DataFrame, max_rows=100_000, verbose=True):
    # Создаем копию DataFrame для безопасной обработки
    DataFrame_clean = DataFrame.copy()
    stats = {'initial_rows': DataFrame_clean.shape[0], 'removed_rows': {}}
    
    if verbose:
        print(f"Исходный размер DataFrame: {DataFrame_clean.shape}")

    # 1. Удаление строк с пропущенными значениями
    initial_rows = DataFrame_clean.shape[0]
    DataFrame_clean = DataFrame_clean.dropna()
    stats['removed_rows']['missing_values'] = initial_rows - DataFrame_clean.shape[0]
    if verbose and stats['removed_rows']['missing_values'] > 0:
        print(f"Удалено {stats['removed_rows']['missing_values']} строк с пропущенными значениями")

    # 2. Проверка и приведение типов данных
    numeric_cols = ['area', 'kitchen_area', 'price', 'level', 'levels', 'days_from_first']
    for col in numeric_cols:
        if col in DataFrame_clean.columns:
            DataFrame_clean[col] = pd.to_numeric(DataFrame_clean[col], errors='coerce')
            DataFrame_clean = DataFrame_clean.dropna(subset=[col])  # Удаляем строки с NaN после приведения
            if verbose:
                print(f"После приведения типов для {col}: {DataFrame_clean.shape}")

    # 3. Удаление строк с отрицательными значениями
    for col in ['area', 'kitchen_area', 'levels', 'days_from_first']:
        if col in DataFrame_clean.columns:
            initial_rows = DataFrame_clean.shape[0]
            DataFrame_clean = DataFrame_clean[DataFrame_clean[col] >= 0]
            stats['removed_rows'][f'negative_{col}'] = initial_rows - DataFrame_clean.shape[0]
            if verbose and stats['removed_rows'][f'negative_{col}'] > 0:
                print(f"Удалено {stats['removed_rows'][f'negative_{col}']} строк с отрицательными значениями в {col}")

    # 4. Фильтрация по разумным пределам площади (20–150 кв.м)
    if 'area' in DataFrame_clean.columns:
        initial_rows = DataFrame_clean.shape[0]
        DataFrame_clean = DataFrame_clean[DataFrame_clean['area'].between(20, 150)]
        stats['removed_rows']['area'] = initial_rows - DataFrame_clean.shape[0]
        if verbose:
            print(f"После фильтрации по площади: {DataFrame_clean.shape}")

    # 5. Фильтрация по разумным пределам цены (0.5 млн – 30 млн)
    if 'price' in DataFrame_clean.columns:
        initial_rows = DataFrame_clean.shape[0]
        DataFrame_clean = DataFrame_clean[DataFrame_clean['price'].between(500_000, 30_000_000)]
        stats['removed_rows']['price'] = initial_rows - DataFrame_clean.shape[0]
        if verbose:
            print(f"После фильтрации по цене: {DataFrame_clean.shape}")

    # 6. Проверка соотношения кухни к общей площади (10–50%)
    if 'kitchen_area' in DataFrame_clean.columns and 'area' in DataFrame_clean.columns:
        initial_rows = DataFrame_clean.shape[0]
        DataFrame_clean = DataFrame_clean[DataFrame_clean['kitchen_area'].between(0.1 * DataFrame_clean['area'], 0.5 * DataFrame_clean['area'])]
        DataFrame_clean = DataFrame_clean[DataFrame_clean['kitchen_area'].between(5, 40)]
        stats['removed_rows']['kitchen_area'] = initial_rows - DataFrame_clean.shape[0]
        if verbose:
            print(f"После фильтрации по площади кухни: {DataFrame_clean.shape}")

    # 7. Проверка, что этаж не превышает общее количество этажей
    if 'level' in DataFrame_clean.columns and 'levels' in DataFrame_clean.columns:
        initial_rows = DataFrame_clean.shape[0]
        DataFrame_clean = DataFrame_clean[DataFrame_clean['level'] <= DataFrame_clean['levels']]
        stats['removed_rows']['level'] = initial_rows - DataFrame_clean.shape[0]
        if verbose:
            print(f"После фильтрации по этажам: {DataFrame_clean.shape}")

    # 8. Ограничение максимального количества этажей (1–30)
    if 'levels' in DataFrame_clean.columns:
        initial_rows = DataFrame_clean.shape[0]
        DataFrame_clean = DataFrame_clean[DataFrame_clean['levels'].between(1, 30)]
        stats['removed_rows']['levels'] = initial_rows - DataFrame_clean.shape[0]
        if verbose:
            print(f"После фильтрации по количеству этажей: {DataFrame_clean.shape}")

    # 9. Фильтрация по количеству комнат (1–7, исключаем 0 для жилых объектов)
    if 'rooms' in DataFrame_clean.columns:
        initial_rows = DataFrame_clean.shape[0]
        DataFrame_clean = DataFrame_clean[DataFrame_clean['rooms'].between(1, 7)]
        stats['removed_rows']['rooms'] = initial_rows - DataFrame_clean.shape[0]
        if verbose:
            print(f"После фильтрации по количеству комнат: {DataFrame_clean.shape}")

    # 10. Удаление выбросов с использованием IQR для числовых признаков
    for col in ['area', 'kitchen_area', 'price']:
        if col in DataFrame_clean.columns:
            initial_rows = DataFrame_clean.shape[0]
            Q1 = DataFrame_clean[col].quantile(0.25)
            Q3 = DataFrame_clean[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            DataFrame_clean = DataFrame_clean[DataFrame_clean[col].between(lower_bound, upper_bound)]
            stats['removed_rows'][f'iqr_{col}'] = initial_rows - DataFrame_clean.shape[0]
            if verbose:
                print(f"После удаления выбросов по {col} (IQR): {DataFrame_clean.shape}")

    # 11. Проверка бинарных признаков (is_Moscow, is_Saint_Petersburg)
    binary_cols = ['is_Moscow', 'is_Saint_Petersburg']
    for col in binary_cols:
        if col in DataFrame_clean.columns:
            initial_rows = DataFrame_clean.shape[0]
            DataFrame_clean = DataFrame_clean[DataFrame_clean[col].isin([0, 1])]
            stats['removed_rows'][f'binary_{col}'] = initial_rows - DataFrame_clean.shape[0]
            if verbose and stats['removed_rows'][f'binary_{col}'] > 0:
                print(f"Удалено {stats['removed_rows'][f'binary_{col}']} строк с некорректными значениями в {col}")

    # 12. Проверка временного признака (0–730 дней, т.е. до 2 лет)
    if 'days_from_first' in DataFrame_clean.columns:
        initial_rows = DataFrame_clean.shape[0]
        DataFrame_clean = DataFrame_clean[DataFrame_clean['days_from_first'].between(0, 730)]
        stats['removed_rows']['days_from_first'] = initial_rows - DataFrame_clean.shape[0]
        if verbose:
            print(f"После фильтрации по 'days_from_first': {DataFrame_clean.shape}")

    # 13. Удаление дублирующихся строк
    initial_rows = DataFrame_clean.shape[0]
    DataFrame_clean = DataFrame_clean.drop_duplicates()
    stats['removed_rows']['duplicates'] = initial_rows - DataFrame_clean.shape[0]
    if verbose and stats['removed_rows']['duplicates'] > 0:
        print(f"Удалено {stats['removed_rows']['duplicates']} дублирующихся строк")

    # 14. Ограничение максимального размера выборки
    if len(DataFrame_clean) > max_rows:
        DataFrame_clean = DataFrame_clean[:max_rows]
        stats['removed_rows']['max_rows_limit'] = len(DataFrame_clean) - max_rows
        if verbose:
            print(f"Ограничено до {max_rows} строк: {DataFrame_clean.shape}")

    # 15. Проверка корреляции признаков с целевой переменной
    if 'price' in DataFrame_clean.columns:
        low_corr_cols = []
        for col in DataFrame_clean.columns:
            if col != 'price' and DataFrame_clean[col].dtype in ['int64', 'float64']:
                corr = DataFrame_clean[col].corr(DataFrame_clean['price'])
                if abs(corr) < 0.1:  # Удаляем признаки с корреляцией менее 0.1
                    low_corr_cols.append(col)
        initial_rows = DataFrame_clean.shape[0]
        DataFrame_clean = DataFrame_clean.drop(columns=low_corr_cols)
        stats['removed_rows']['low_correlation'] = initial_rows - DataFrame_clean.shape[0]
        if verbose and low_corr_cols:
            print(f"Удалены столбцы с низкой корреляцией с ценой: {low_corr_cols}")

    if verbose:
        print(f"Итоговый размер DataFrame: {DataFrame_clean.shape}")
        print(f"Статистика удаленных строк: {stats['removed_rows']}")

    return DataFrame_clean, stats

# Применение супер-улучшенной очистки
DataFrame_clean, cleaning_stats = super_clean_dataframe(DataFrame)

Исходный размер DataFrame: (11358150, 14)
После приведения типов для area: (11358150, 14)
После приведения типов для kitchen_area: (11358150, 14)
После приведения типов для price: (11358150, 14)
После приведения типов для days_from_first: (11358150, 14)
Удалено 1092246 строк с отрицательными значениями в kitchen_area
После фильтрации по площади: (9995310, 14)
После фильтрации по цене: (9845998, 14)
После фильтрации по площади кухни: (6692759, 14)
После фильтрации по количеству комнат: (6533534, 14)
После удаления выбросов по area (IQR): (6342256, 14)
После удаления выбросов по kitchen_area (IQR): (6030608, 14)
После удаления выбросов по price (IQR): (5597813, 14)
После фильтрации по 'days_from_first': (5597813, 14)
Удалено 177780 дублирующихся строк
Ограничено до 100000 строк: (100000, 14)
Удалены столбцы с низкой корреляцией с ценой: ['building_type_1', 'building_type_2', 'building_type_5', 'building_type_6', 'days_from_first', 'level_ratio']
Итоговый размер DataFrame: (100000, 8)
Ста

In [132]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression as SkLinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

def run_final_experiments():
    print("=== ФИНАЛЬНЫЕ ЭКСПЕРИМЕНТЫ С МОДЕЛЯМИ ===")

    SAMPLE_SIZE = 100_000
    TEST_SIZE = 20_000

    # def strict_clean_dataframe(DataFrame):
    #     DataFrame_clean = DataFrame.copy()
        
    #     # Оставляем только числовые столбцы
    #     numeric_cols = DataFrame_clean.select_dtypes(include=[np.number]).columns
    #     DataFrame_clean = DataFrame_clean[numeric_cols]
        
    #     # Фильтрация выбросов
    #     DataFrame_clean = DataFrame_clean[DataFrame_clean['area'].between(25, 150)]
    #     DataFrame_clean = DataFrame_clean[DataFrame_clean['price'].between(1_000_000, 20_000_000)]
        
    #     if 'kitchen_area' in DataFrame_clean.columns:
    #         DataFrame_clean = DataFrame_clean[DataFrame_clean['kitchen_area'].between(5, 50)]
    #     if 'level_ratio' in DataFrame_clean.columns:
    #         DataFrame_clean = DataFrame_clean[DataFrame_clean['level_ratio'].between(0.05, 0.95)]
    #     if 'rooms' in DataFrame_clean.columns:
    #         DataFrame_clean = DataFrame_clean[DataFrame_clean['rooms'].between(1, 5)]
    #     if 'days_from_first' in DataFrame_clean.columns:
    #         DataFrame_clean = DataFrame_clean[DataFrame_clean['days_from_first'].between(0, 1000)]

    #     # Ограничение размера выборки
    #     DataFrame_clean = DataFrame_clean[:SAMPLE_SIZE]
        
    #     # Удаление выбросов по целевой переменной (цене) на основе квантилей
    #     target = 'price'
    #     q_low, q_high = DataFrame_clean[target].quantile([0.05, 0.95])
    #     DataFrame_clean = DataFrame_clean[DataFrame_clean[target].between(q_low, q_high)]
        
    #     return DataFrame_clean

    # Применяем очистку данных
    DataFrame_strict = DataFrame_clean
    print(f"Размер данных после очистки: {DataFrame_strict.shape}")

    # Подготовка признаков и целевой переменной
    X_final = DataFrame_strict.drop(columns=['price'])
    y_final = DataFrame_strict['price']
    
    # Оставляем только числовые признаки
    X_final = X_final.select_dtypes(include=[np.number])
    print(f"Числовые столбцы: {X_final.columns.tolist()}")

    # Создаем подвыборку
    sample_indices = np.random.choice(len(X_final), size=min(SAMPLE_SIZE + TEST_SIZE, len(X_final)), replace=False)
    X_sample = X_final.iloc[sample_indices]
    y_sample = y_final.iloc[sample_indices]

    # Разделение на обучающую и тестовую выборки
    X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(
        X_sample, y_sample, test_size=TEST_SIZE, random_state=42
    )

    print(f"Размер обучающей выборки: {X_train_final.shape}")
    print(f"Размер тестовой выборки: {X_test_final.shape}")
    print(f"Диапазон цен: {y_train_final.min():,.0f} - {y_train_final.max():,.0f}\n")

    def safe_metrics(y_true, y_pred, model_name):
        y_true = np.array(y_true, dtype=np.float64)
        y_pred = np.array(y_pred, dtype=np.float64)
        
        # Защита от NaN и экстремальных значений
        y_pred = np.nan_to_num(y_pred, nan=np.mean(y_true))
        y_pred = np.clip(y_pred, y_true.min() * 0.1, y_true.max() * 10)
        
        mse = mean_squared_error(y_true, y_pred)
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        
        # Вычисление MAPE для значимых значений
        mask = (y_true != 0) & (np.abs(y_true) > 1000)
        mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100 if np.sum(mask) > len(y_true) * 0.5 else np.inf
        
        print(f"{model_name} - MSE: {mse:.2f}, MAE: {mae:.2f}, RMSE: {rmse:.2f}, MAPE: {mape:.1f}%")
        return [mse, mae, rmse, mape]

    results = []

    # 1. Обучение собственной LinearRegression (SGD)
    print("1. Обучение LinearRegression (SGD)...")
    try:
        lr_sgd = LinearRegression(learning_rate=0.1, optimization='SGD', max_iter=300, tolerance=1e-4)
        lr_sgd.fit(X_train_final, y_train_final)
        y_pred_sgd = lr_sgd.predict(X_test_final)
        
        metrics_sgd = safe_metrics(y_test_final, y_pred_sgd, "Our LR (SGD)")
        results.append(["Our LR (SGD)"] + metrics_sgd)
        print(f"   ✅ Успешно. Финальная ошибка: {lr_sgd.loss_history[-1]:.4f}\n")
    except Exception as e:
        print(f"   ❌ Ошибка: {e}")
        results.append(["Our LR (SGD)", np.inf, np.inf, np.inf, np.inf])

    # 2. Обучение собственной LinearRegression (Momentum)
    print("2. Обучение LinearRegression (Momentum)...")
    try:
        lr_momentum = LinearRegression(learning_rate=0.1, optimization='Momentum', max_iter=300, tolerance=1e-4)
        lr_momentum.fit(X_train_final, y_train_final)
        y_pred_momentum = lr_momentum.predict(X_test_final)
        
        metrics_momentum = safe_metrics(y_test_final, y_pred_momentum, "Our LR (Momentum)")
        results.append(["Our LR (Momentum)"] + metrics_momentum)
        print(f"   ✅ Успешно. Финальная ошибка: {lr_momentum.loss_history[-1]:.4f}\n")
    except Exception as e:
        print(f"   ❌ Ошибка: {e}")
        results.append(["Our LR (Momentum)", np.inf, np.inf, np.inf, np.inf])

    # 3. Обучение собственной LinearRegression (AdaGrad)
    print("3. Обучение LinearRegression (AdaGrad)...")
    try:
        lr_adagrad = LinearRegression(learning_rate=0.1, optimization='AdaGrad', max_iter=300, tolerance=1e-4)
        lr_adagrad.fit(X_train_final, y_train_final)
        y_pred_adagrad = lr_adagrad.predict(X_test_final)
        
        metrics_adagrad = safe_metrics(y_test_final, y_pred_adagrad, "Our LR (AdaGrad)")
        results.append(["Our LR (AdaGrad)"] + metrics_adagrad)
        print(f"   ✅ Успешно. Финальная ошибка: {lr_adagrad.loss_history[-1]:.4f}\n")
    except Exception as e:
        print(f"   ❌ Ошибка: {e}")
        results.append(["Our LR (AdaGrad)", np.inf, np.inf, np.inf, np.inf])

    # 4. Обучение библиотечной LinearRegression
    print("4. Обучение библиотечной LinearRegression...")
    try:
        lr_sklearn = SkLinearRegression()
        lr_sklearn.fit(X_train_final, y_train_final)
        y_pred_sklearn = lr_sklearn.predict(X_test_final)
        
        metrics_sklearn = safe_metrics(y_test_final, y_pred_sklearn, "Sklearn LR")
        results.append(["Sklearn LinearRegression"] + metrics_sklearn)
        print("   ✅ Успешно\n")
    except Exception as e:
        print(f"   ❌ Ошибка: {e}")
        results.append(["Sklearn LinearRegression", np.inf, np.inf, np.inf, np.inf])

    # 5. Обучение собственного KNNRegressor
    print("5. Обучение KNNRegressor...")
    try:
        knn_our = KNNRegressor(n_neighbors=3)
        knn_our.fit(X_train_final, y_train_final)
        y_pred_knn = knn_our.predict(X_test_final)
        
        metrics_knn = safe_metrics(y_test_final, y_pred_knn, "Our KNN")
        results.append(["Our KNN"] + metrics_knn)
        print("   ✅ Успешно\n")
    except Exception as e:
        print(f"   ❌ Ошибка: {e}")
        results.append(["Our KNN", np.inf, np.inf, np.inf, np.inf])

    # 6. Обучение библиотечного KNeighborsRegressor
    print("6. Обучение библиотечного KNeighborsRegressor...")
    try:
        knn_sklearn = KNeighborsRegressor(n_neighbors=5)
        knn_sklearn.fit(X_train_final, y_train_final)
        y_pred_sklearn_knn = knn_sklearn.predict(X_test_final)
        
        metrics_sklearn_knn = safe_metrics(y_test_final, y_pred_sklearn_knn, "Sklearn KNN")
        results.append(["Sklearn KNN"] + metrics_sklearn_knn)
        print("   ✅ Успешно\n")
    except Exception as e:
        print(f"   ❌ Ошибка: {e}")
        results.append(["Sklearn KNN", np.inf, np.inf, np.inf, np.inf])

    # Формирование и вывод результатов
    print("="*80)
    print("ИТОГОВЫЕ РЕЗУЛЬТАТЫ:")
    print("="*80)

    results_DataFrame = pd.DataFrame(results, columns=["Model", "MSE", "MAE", "RMSE", "MAPE"])

    def format_metric(value):
        if value == np.inf:
            return "FAILED"
        elif value > 1e12:
            return f"{value/1e12:.2f}T"
        elif value > 1e9:
            return f"{value/1e9:.2f}B"
        elif value > 1e6:
            return f"{value/1e6:.2f}M"
        elif value > 1e3:
            return f"{value/1e3:.2f}K"
        else:
            return f"{value:.2f}"

    display_DataFrame = results_DataFrame.copy()
    for col in ['MSE', 'MAE', 'RMSE']:
        display_DataFrame[col] = display_DataFrame[col].apply(format_metric)
    display_DataFrame['MAPE'] = results_DataFrame['MAPE'].apply(lambda x: f"{x:.1f}%" if x != np.inf else "FAILED")

    print(display_DataFrame.to_string(index=False))

    # Анализ результатов
    working_models = results_DataFrame[results_DataFrame['MSE'] != np.inf]
    if not working_models.empty:
        best_mse_idx = working_models['MSE'].idxmin()
        best_model = working_models.loc[best_mse_idx, 'Model']
        best_mse = working_models.loc[best_mse_idx, 'MSE']
        
        print(f"\n🏆 Лучшая модель: {best_model} (MSE: {format_metric(best_mse)})")
        
        our_success = [m for m in working_models['Model'] if 'Our' in m]
        sklearn_success = [m for m in working_models['Model'] if 'Sklearn' in m]
        
        print(f"✅ Наши рабочие модели: {', '.join(our_success) if our_success else 'НЕТ'}")
        print(f"✅ Библиотечные модели: {', '.join(sklearn_success)}")
    else:
        print("\n❌ Все модели не сработали")

    return results_DataFrame

# Запуск финальных экспериментов
final_results = run_final_experiments()

=== ФИНАЛЬНЫЕ ЭКСПЕРИМЕНТЫ С МОДЕЛЯМИ ===
Размер данных после очистки: (100000, 8)
Числовые столбцы: ['rooms', 'area', 'kitchen_area', 'building_type_3', 'building_type_4']
Размер обучающей выборки: (80000, 5)
Размер тестовой выборки: (20000, 5)
Диапазон цен: 500,000 - 11,434,020

1. Обучение LinearRegression (SGD)...
Our LR (SGD) - MSE: 3643864300929.82, MAE: 1402324.56, RMSE: 1908890.86, MAPE: 47.7%
   ✅ Успешно. Финальная ошибка: 0.7428

2. Обучение LinearRegression (Momentum)...
Our LR (Momentum) - MSE: 3728181380551.85, MAE: 1411560.56, RMSE: 1930849.91, MAPE: 46.4%
   ✅ Успешно. Финальная ошибка: 0.7617

3. Обучение LinearRegression (AdaGrad)...
Our LR (AdaGrad) - MSE: 3671043695412.88, MAE: 1398643.60, RMSE: 1915996.79, MAPE: 47.3%
   ✅ Успешно. Финальная ошибка: 0.7539

4. Обучение библиотечной LinearRegression...
Sklearn LR - MSE: 3640003978992.04, MAE: 1400509.13, RMSE: 1907879.45, MAPE: 47.4%
   ✅ Успешно

5. Обучение KNNRegressor...
Our KNN - MSE: 3641152203699.41, MAE: 133

In [ ]:
import numpy as np
import pandas as pd
from numba import njit, prange
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

@njit(parallel=True, fastmath=True)
def calculate_euclidean_distances(X1, X2):
    n1, n2 = X1.shape[0], X2.shape[0]
    distances = np.empty((n1, n2), dtype=np.float32)
    for i in prange(n1):
        for j in range(n2):
            sum_squares = 0.0
            for k in range(X1.shape[1]):
                diff = X1[i, k] - X2[j, k]
                sum_squares += diff * diff
            distances[i, j] = np.sqrt(sum_squares)
    return distances

class FastKNNRegressor:
    def __init__(self, n_neighbors=3, batch_size=5000):
        self.k = n_neighbors
        self.batch_size = batch_size

    def fit(self, X, y):
        self.X_train = X.to_numpy(dtype=np.float32) if not isinstance(X, np.ndarray) else X.astype(np.float32)
        self.y_train = y.to_numpy(dtype=np.float32) if not isinstance(y, np.ndarray) else y.astype(np.float32)

    def predict(self, X):
        X = X.to_numpy(dtype=np.float32) if not isinstance(X, np.ndarray) else X.astype(np.float32)
        n_samples = X.shape[0]
        predictions = np.zeros(n_samples, dtype=np.float32)

        for start in range(0, n_samples, self.batch_size):
            end = min(start + self.batch_size, n_samples)
            X_batch = X[start:end]

            # Вычисляем расстояния для текущего батча
            distances = calculate_euclidean_distances(X_batch, self.X_train)
            
            # Находим k ближайших соседей
            indices = np.argpartition(distances, self.k, axis=1)[:, :self.k]
            nearest_distances = np.take_along_axis(distances, indices, axis=1)
            nearest_values = self.y_train[indices]

            # Вычисляем веса как 1 / (расстояние + эпсилон)
            weights = 1.0 / (nearest_distances + 1e-6)
            weighted_avg = np.sum(nearest_values * weights, axis=1) / np.sum(weights, axis=1)
            predictions[start:end] = weighted_avg

        return predictions

print("✅ Подготовка данных из DataFrame")
DataFrame = DataFrame_clean  # Работаем с очищенным DataFrame
DataFrame = DataFrame[:100_000]  # Ограничиваем размер выборки

target = "price"
features = [col for col in DataFrame.columns if col != target]

# Удаляем выбросы по целевой переменной на основе квантилей
q_low, q_high = DataFrame[target].quantile([0.05, 0.95])
DataFrame = DataFrame[DataFrame[target].between(q_low, q_high)]

# Применяем логарифмирование к целевой переменной
DataFrame[target] = np.log1p(DataFrame[target])

# Масштабируем признаки
scaler = StandardScaler()
X_scaled = scaler.fit_transform(DataFrame[features])
y = DataFrame[target].values

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=28
)

print("\n🔹 Обучение FastKNNRegressor...")
knn_model = FastKNNRegressor(n_neighbors=3, batch_size=200)
knn_model.fit(X_train, y_train)

print("🔹 Выполнение предсказаний...")
y_pred_log = knn_model.predict(X_test)

# Обратное преобразование предсказаний и истинных значений из логарифма
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)

mape = mean_absolute_percentage_error(y_true, y_pred) * 100
print(f"\n✅ FastKNNRegressor завершен. MAPE = {mape:.2f}%")

# Формируем таблицу с примерами предсказаний
results_DataFrame = pd.DataFrame({
    "y_true": y_true[:10],
    "y_pred": y_pred[:10],
    "error_%": np.abs(y_true[:10] - y_pred[:10]) / y_true[:10] * 100
})

print("\nПримеры первых 10 предсказаний:")
print(results_DataFrame)

# Возвращаем результаты для дальнейшего анализа
results_DataFrame

✅ Подготовка данных из DataFrame

🔹 Обучение FastKNNRegressor...
🔹 Выполнение предсказаний...

✅ FastKNNRegressor завершен. MAPE = 29.76%

Примеры первых 10 предсказаний:
      y_true      y_pred     error_%
0  1350000.0  2369222.75   75.497981
1  3500000.0  2229375.00   36.303571
2  5600000.0  5599879.00    0.002161
3  4350000.0  5129351.50   17.916126
4  2167000.0  2073654.75    4.307580
5  1750000.0  3595983.50  105.484771
6  4050000.0  2147552.25   46.974019
7  3100000.0  2247552.50   27.498306
8  6750000.0  6674527.00    1.118119
9  7566460.0  7437102.00    1.709624


,y_true,y_pred,error_%
0,1350000.0,2369222.75,75.497981
1,3500000.0,2229375.00,36.303571
2,5600000.0,5599879.00,0.002161
3,4350000.0,5129351.50,17.916126
4,2167000.0,2073654.75,4.307580
5,1750000.0,3595983.50,105.484771
6,4050000.0,2147552.25,46.974019
7,3100000.0,2247552.50,27.498306
8,6750000.0,6674527.00,1.118119
9,7566460.0,7437102.00,1.709624
